In [ ]:
# the aim of this notebook is to end up with the samples of interest into a pickles
# look at voice first then whale

In [ ]:
# voice

In [1]:
import sys
sys.path.append("/Users/david/Documents/code/fhmm/v1")

In [2]:
import src.ads_davidjames9610.ads as ads
import src.ads_davidjames9610.useful as useful
import numpy as np
import os
import os.path
from pathlib import Path
import matplotlib.pyplot as plt
from importlib import reload
from src.classifiers_davidjames9610.test_a.e_config import *

In [3]:
datasetFolder = r"/Users/david/Documents/data/speechCommands"
speech_ads = ads.AudioDatastore()
speech_ads.populate(datasetFolder,include_sub_folders=True, label_source=True)

In [4]:
unique_words = np.unique(speech_ads.labels)

dict_of_words = {}
arr_of_speakers = []
dict_of_speakers = {}

for word in unique_words:
    ads_subset = ads.subset(speech_ads, label=word)
    speakers = []
    for file in ads_subset.files:
        nm = os.path.basename(file)
        nm = nm.split('_')[0]
        speakers.append('a' + nm)
    unique_labels, unique_counts = np.unique(speakers, return_counts=True)
    filtered_labels = (unique_labels[unique_counts > 8])
    if len(filtered_labels) > 1:
        dict_of_words[word] = filtered_labels
        for label in filtered_labels:
            if label in dict_of_speakers:
                dict_of_speakers[label].append(word)
            else:
                dict_of_speakers[label] = [word]

speakers_dense = []
for key in dict_of_speakers:
    noises = dict_of_speakers[key]
    if len(noises) > 7 and (key not in ['ac50f55b8']):
        speakers_dense.append(key)

speakers = []
for file in speech_ads.files:
    nm = os.path.basename(file)
    nm = nm.split('_')[0]
    speakers.append('a' + nm)

speech_ads.set(labels=speakers)

ads_all = []
for speaker in speakers_dense:
    ads_all.append(ads.subset(speech_ads, label=speaker))

In [5]:
speakers = []
for file in speech_ads.files:
    nm = os.path.basename(file)
    nm = nm.split('_')[0]
    speakers.append('a' + nm)

speech_ads.set(labels=speakers)

In [6]:
ads_all = []
for speaker in speakers_dense:
    ads_all.append(ads.subset(speech_ads, label=speaker))

In [7]:
# possibly updated this to include a better VAD
samples = []
labels = []

labels_set = speakers_dense
num_to_label = {i: labels_set[i] for i in range(len(labels_set))}
label_to_num = {labels_set[i] : i for i in range(len(labels_set))}

count = 0
for speaker_ads in ads_all:
    count +=1
    if count % 10 == 0: print(count)
    labels.extend([label_to_num[label] for label in speaker_ads.labels]) # all the same
    samples.extend([useful.file_to_audio(file, sr)[0] for file in speaker_ads.files])

In [8]:
import pickle
pickle.dump({
    'samples': samples,
    'labels': labels,
    'labels_set': labels_set,
    'num_to_label': num_to_label,
    'label_to_num': label_to_num
}, open('samples_labels.pickle','wb'))